## Pre-process scraped data

In [2]:
import pandas as pd 
from collections import Counter
import re

### 1. Construct dataframe of all songs

In [3]:
import os 
genres = ['Country', 'Jazz', 'Pop', 'Rock']
all_genres = pd.DataFrame()
for genre in genres:
    basepath = f'./scrapped_data/{genre}/'
    filepaths = [basepath + f for f in os.listdir(f'./scrapped_data/{genre}/') if f.endswith('.csv')]
    genre_df = pd.concat(map(pd.read_csv, filepaths))
    genre_df['genre'] = genre
    all_genres = pd.concat([all_genres, genre_df], ignore_index=True).drop(columns='Unnamed: 0')
all_genres

,name,author,link,lyrics,chords,genre
0,Ring Of Fire,Johnny Cash,https://tabs.ultimate-guitar.com/tab/johnny-ca...,Love Is A Burning Thing\r\nAnd It Makes A Fier...,"['G', 'C', 'G', 'C', 'G', 'D', 'G', 'D', 'G', ...",Country
1,Leaving On A Jet Plane,John Denver,https://tabs.ultimate-guitar.com/tab/john-denv...,"All my bags are packed, I'm ready to go\r\nI'm...","['G', 'C', 'G', 'C', 'G', 'C', 'D', 'G', 'C', ...",Country
2,Hit The Road Jack,Ray Charles,https://tabs.ultimate-guitar.com/tab/ray-charl...,Hit the road Jack and don't you come back no m...,"['Am', 'G', 'F', 'E7', 'Am', 'G', 'F', 'E7', '...",Country
3,The End Of The World,Skeeter Davis,https://tabs.ultimate-guitar.com/tab/skeeter-d...,Why does the sun go on shining\r\nWhy does the...,"['G', 'D', 'Em', 'Bm', 'Am', 'Bm', 'E7', 'Am',...",Country
4,Mama Tried,Merle Haggard,https://tabs.ultimate-guitar.com/tab/merle-hag...,The first thing I remember knowing was a lones...,"['D', 'G', 'D', 'G', 'D', 'A7', 'D', 'G', 'D',...",Country
...,...,...,...,...,...,...
27046,Fake Love Dont Last,Machine Gun Kelly,https://tabs.ultimate-guitar.com/tab/machine-g...,"I watched a movie scene, got déjà vu\r\n'Cause...","['Em', 'G', 'B', 'C', 'Em', 'G', 'B', 'C', 'Em...",Rock
27047,Sid And Nancy,Machine Gun Kelly,https://tabs.ultimate-guitar.com/tab/machine-g...,I knew a girl who'd wear my t-shirts when she ...,"['D', 'F#m', 'E', 'D', 'F#m', 'E', 'D', 'F#m',...",Rock
27048,Forever,Stereophonics,https://tabs.ultimate-guitar.com/tab/stereopho...,Sun beats down on my mind on a friday morning...,"['G', 'C', 'G', 'C', 'G', 'C', 'G', 'C', 'D', ...",Rock
27049,Goodnight Chicago,Rainbow Kitten Surprise,https://tabs.ultimate-guitar.com/tab/rainbow-k...,Twenty years to see New York reflected on subw...,"['C', 'C', 'Am', 'F', 'C', 'C', 'C', 'Am', 'F'...",Rock


### 2. Remove duplicates

Parse real name of a song i.e. remove the version. Notice song on index 7 and 8.

In [4]:
all_genres['actual_name'] = all_genres.name.apply(lambda name : name.split(' (ver')[0])
all_genres.head(10)

,name,author,link,lyrics,chords,genre,actual_name
0,Ring Of Fire,Johnny Cash,https://tabs.ultimate-guitar.com/tab/johnny-ca...,Love Is A Burning Thing\r\nAnd It Makes A Fier...,"['G', 'C', 'G', 'C', 'G', 'D', 'G', 'D', 'G', ...",Country,Ring Of Fire
1,Leaving On A Jet Plane,John Denver,https://tabs.ultimate-guitar.com/tab/john-denv...,"All my bags are packed, I'm ready to go\r\nI'm...","['G', 'C', 'G', 'C', 'G', 'C', 'D', 'G', 'C', ...",Country,Leaving On A Jet Plane
2,Hit The Road Jack,Ray Charles,https://tabs.ultimate-guitar.com/tab/ray-charl...,Hit the road Jack and don't you come back no m...,"['Am', 'G', 'F', 'E7', 'Am', 'G', 'F', 'E7', '...",Country,Hit The Road Jack
3,The End Of The World,Skeeter Davis,https://tabs.ultimate-guitar.com/tab/skeeter-d...,Why does the sun go on shining\r\nWhy does the...,"['G', 'D', 'Em', 'Bm', 'Am', 'Bm', 'E7', 'Am',...",Country,The End Of The World
4,Mama Tried,Merle Haggard,https://tabs.ultimate-guitar.com/tab/merle-hag...,The first thing I remember knowing was a lones...,"['D', 'G', 'D', 'G', 'D', 'A7', 'D', 'G', 'D',...",Country,Mama Tried
5,Cocaine Blues,Johnny Cash,https://tabs.ultimate-guitar.com/tab/johnny-ca...,"Early one mornin' while makin' the rounds, I t...","['C', 'G', 'C', 'C', 'G', 'C', 'C', 'G', 'C', ...",Country,Cocaine Blues
6,King Of The Road (ver 4),Roger Miller,https://tabs.ultimate-guitar.com/tab/roger-mil...,"Trailers for sale or rent,\r\nRooms to let fif...","['A', 'D', 'E', 'A', 'A', 'D', 'E', 'A', 'D', ...",Country,King Of The Road
7,Jackson,Johnny Cash,https://tabs.ultimate-guitar.com/tab/johnny-ca...,We got married in a fever hotter than a pepper...,"['C', 'C', 'C7', 'C', 'F', 'C', 'C', 'F', 'G7'...",Country,Jackson
8,Crazy (ver 2),Patsy Cline,https://tabs.ultimate-guitar.com/tab/patsy-cli...,"Crazy, I'm crazy for feeling so lonely\r\nI'm ...","['C', 'C', 'A7', 'A7', 'Dm', 'Ebm7', 'Dm7', 'D...",Country,Crazy
9,Lay Lady Lay,Bob Dylan,https://tabs.ultimate-guitar.com/tab/bob-dylan...,"Lay lady lay, lay across my big brass bed\...","['A', 'C#m', 'G', 'Bm', 'A', 'C#m', 'G', 'Bm',...",Country,Lay Lady Lay


In [5]:
print(all_genres.actual_name.value_counts())
all_genres.loc[all_genres.actual_name.str.contains('What A Wonderful World')].head(10)

What A Wonderful World       39
My Way                       22
Hallelujah                   20
Georgia On My Mind           19
I Cant Make You Love Me      18
                             ..
Eternity Road                 1
All I Needed Was The Rain     1
Kissin On The Phone           1
Two Ragged Soldiers           1
Goodnight Chicago             1
Name: actual_name, Length: 17068, dtype: int64


,name,author,link,lyrics,chords,genre,actual_name
2157,What A Wonderful World,Stacey Kent,https://tabs.ultimate-guitar.com/tab/stacey-ke...,"I see trees of green, red roses too\r\n ...","['A', 'C#m7', 'Dmaj7', 'C#m7', 'Bm7', 'A', 'C#...",Jazz,What A Wonderful World
2364,What A Wonderful World (ver 9),Louis Armstrong,https://tabs.ultimate-guitar.com/tab/louis-arm...,"I see trees of green, red roses too\r\nI see t...","['C', 'Em', 'F', 'Em', 'Dm', 'C', 'E7', 'Am', ...",Jazz,What A Wonderful World
2369,What A Wonderful World (ver 2),Louis Armstrong,https://tabs.ultimate-guitar.com/tab/louis-arm...,"I see trees of green, red roses too\r\n I...","['F', 'Am', 'Bb', 'Am', 'Gm7', 'F', 'A7', 'Dm'...",Jazz,What A Wonderful World
2379,What A Wonderful World,Louis Armstrong,https://tabs.ultimate-guitar.com/tab/louis-arm...,"I see trees of green, red roses too\r\n ...","['F', 'Am', 'Bb', 'Am', 'Gm', 'F', 'A7', 'Dm',...",Jazz,What A Wonderful World
2392,What A Wonderful World (ver 3),Louis Armstrong,https://tabs.ultimate-guitar.com/tab/louis-arm...,"I see trees of green, red roses too\r\nI see t...","['G', 'Bm', 'C', 'Bm', 'Am7', 'G', 'B7', 'Em',...",Jazz,What A Wonderful World
2405,What A Wonderful World (ver 11),Louis Armstrong,https://tabs.ultimate-guitar.com/tab/louis-arm...,"I see trees of green, red roses too\r\nI see t...","['C', 'G', 'Am', 'Em', 'Dm', 'C', 'E7', 'Am', ...",Jazz,What A Wonderful World
2409,What A Wonderful World (ver 4),Louis Armstrong,https://tabs.ultimate-guitar.com/tab/louis-arm...,"I see trees of green, red roses too\r\nI see t...","['F', 'Am', 'Bb', 'Am', 'Gm7', 'F', 'A7', 'Dm'...",Jazz,What A Wonderful World
2508,What A Wonderful World,Willie Nelson,https://tabs.ultimate-guitar.com/tab/willie-ne...,"I see trees of green, red roses too\r\nI see t...","['D', 'F#m', 'G', 'F#m', 'Em', 'D', 'F#7', 'Bm...",Jazz,What A Wonderful World
2710,What A Wonderful World (ver 14),Louis Armstrong,https://tabs.ultimate-guitar.com/tab/louis-arm...,"I see trees of green, red roses too,\r\nI see ...","['C', 'G', 'Am', 'Em', 'F', 'Em', 'Dm', 'Am', ...",Jazz,What A Wonderful World
2721,What A Wonderful World (ver 7),Louis Armstrong,https://tabs.ultimate-guitar.com/tab/louis-arm...,"I see trees of green, red roses too\r\nI see t...","['C', 'Em', 'F', 'Em', 'Dm7', 'C', 'E7', 'Am',...",Jazz,What A Wonderful World


Remove duplicate songs that have the same name, have same author, but in different versions.

In [34]:
all_genres_unique = all_genres.drop_duplicates(subset=['actual_name', 'author', 'genre']).reset_index()
all_genres_unique

,index,name,author,link,lyrics,chords,genre,actual_name
0,0,Ring Of Fire,Johnny Cash,https://tabs.ultimate-guitar.com/tab/johnny-ca...,Love Is A Burning Thing\r\nAnd It Makes A Fier...,"['G', 'C', 'G', 'C', 'G', 'D', 'G', 'D', 'G', ...",Country,Ring Of Fire
1,1,Leaving On A Jet Plane,John Denver,https://tabs.ultimate-guitar.com/tab/john-denv...,"All my bags are packed, I'm ready to go\r\nI'm...","['G', 'C', 'G', 'C', 'G', 'C', 'D', 'G', 'C', ...",Country,Leaving On A Jet Plane
2,2,Hit The Road Jack,Ray Charles,https://tabs.ultimate-guitar.com/tab/ray-charl...,Hit the road Jack and don't you come back no m...,"['Am', 'G', 'F', 'E7', 'Am', 'G', 'F', 'E7', '...",Country,Hit The Road Jack
3,3,The End Of The World,Skeeter Davis,https://tabs.ultimate-guitar.com/tab/skeeter-d...,Why does the sun go on shining\r\nWhy does the...,"['G', 'D', 'Em', 'Bm', 'Am', 'Bm', 'E7', 'Am',...",Country,The End Of The World
4,4,Mama Tried,Merle Haggard,https://tabs.ultimate-guitar.com/tab/merle-hag...,The first thing I remember knowing was a lones...,"['D', 'G', 'D', 'G', 'D', 'A7', 'D', 'G', 'D',...",Country,Mama Tried
...,...,...,...,...,...,...,...,...
20219,27045,5150,Machine Gun Kelly,https://tabs.ultimate-guitar.com/tab/machine-g...,Bruises don't heal overnight\r\nI'm a few sips...,"['C', 'G', 'D', 'Em', 'C', 'G', 'D', 'Em', 'C'...",Rock,5150
20220,27046,Fake Love Dont Last,Machine Gun Kelly,https://tabs.ultimate-guitar.com/tab/machine-g...,"I watched a movie scene, got déjà vu\r\n'Cause...","['Em', 'G', 'B', 'C', 'Em', 'G', 'B', 'C', 'Em...",Rock,Fake Love Dont Last
20221,27047,Sid And Nancy,Machine Gun Kelly,https://tabs.ultimate-guitar.com/tab/machine-g...,I knew a girl who'd wear my t-shirts when she ...,"['D', 'F#m', 'E', 'D', 'F#m', 'E', 'D', 'F#m',...",Rock,Sid And Nancy
20222,27048,Forever,Stereophonics,https://tabs.ultimate-guitar.com/tab/stereopho...,Sun beats down on my mind on a friday morning...,"['G', 'C', 'G', 'C', 'G', 'C', 'G', 'C', 'D', ...",Rock,Forever


Notice some songs are performed by different artists and should also be removed based on the lyrics I guess. See example:

In [35]:
print(all_genres_unique.actual_name.value_counts())
all_genres_unique.loc[all_genres_unique.actual_name.str.contains('What a wonderful world', flags=re.IGNORECASE)].head(10)

Home                           12
You And I                      11
Run                            11
Someday                        11
Heaven                         10
                               ..
Vu De Lexterieur                1
Aéroplanes                      1
Je Suis Une Bande De Jeunes     1
Serasa                          1
Goodnight Chicago               1
Name: actual_name, Length: 17068, dtype: int64


,index,name,author,link,lyrics,chords,genre,actual_name
1778,2157,What A Wonderful World,Stacey Kent,https://tabs.ultimate-guitar.com/tab/stacey-ke...,"I see trees of green, red roses too\r\n ...","['A', 'C#m7', 'Dmaj7', 'C#m7', 'Bm7', 'A', 'C#...",Jazz,What A Wonderful World
1978,2364,What A Wonderful World (ver 9),Louis Armstrong,https://tabs.ultimate-guitar.com/tab/louis-arm...,"I see trees of green, red roses too\r\nI see t...","['C', 'Em', 'F', 'Em', 'Dm', 'C', 'E7', 'Am', ...",Jazz,What A Wonderful World
2077,2508,What A Wonderful World,Willie Nelson,https://tabs.ultimate-guitar.com/tab/willie-ne...,"I see trees of green, red roses too\r\nI see t...","['D', 'F#m', 'G', 'F#m', 'Em', 'D', 'F#7', 'Bm...",Jazz,What A Wonderful World
2626,3239,What A Wonderful World,Eva Cassidy,https://tabs.ultimate-guitar.com/tab/eva-cassi...,"I see trees that are green, and red roses too...","['D', 'D7M', 'G', 'D', 'Em', 'D', 'F#', 'Bm', ...",Jazz,What A Wonderful World
3448,4312,What A Wonderful World,Jon Batiste,https://tabs.ultimate-guitar.com/tab/jon-batis...,"I see trees of green, red roses too\r\nI see t...","['E', 'A', 'E', 'Asus2', 'Asus2', 'E', 'C#m', ...",Jazz,What A Wonderful World
3695,4634,What A Wonderful World (ver 5),Louis Armstrong,https://tabs.ultimate-guitar.com/tab/louis-arm...,I see trees of green red roses too\r\nI see th...,"['G', 'D', 'Am7', 'G', 'C', 'G', 'D', 'A7sus',...",Pop,What A Wonderful World
8516,10694,Somewhere Over The Rainbow What A Wonderful World,Robin Schulz,https://tabs.ultimate-guitar.com/tab/robin-sch...,Somewhere over the rainbow\r\n Bluebirds fly\...,"['C', 'Em', 'F', 'C', 'F', 'C', 'G', 'Am', 'F'...",Pop,Somewhere Over The Rainbow What A Wonderful World
18807,24482,What A Wonderful World,Joey Ramone,https://tabs.ultimate-guitar.com/tab/joey-ramo...,"I see trees of green / Red roses, too / I see ...","['F', 'Am', 'Bb', 'Am', 'Gm', 'F', 'A7', 'Dm',...",Rock,What A Wonderful World


### 3. Remove non-English songs

In [36]:
from langdetect import detect_langs, detect

def get_lang(text):
    print(f'{text} -- {detect_langs(text)}')
    print(detect(text))
get_lang('Hello')
get_lang('hi what is up')
get_lang('bonjour comment ca va')
get_lang('hej vad gör du här ens')

Hello -- [fi:0.560895950795585, no:0.4391040492044151]
fi
hi what is up -- [en:0.9999986492706422]
en
bonjour comment ca va -- [fr:0.9999939489780993]
fr
hej vad gör du här ens -- [sv:0.9999988241205873]
sv


In [39]:
print(all_genres_unique.iloc[0, :].lyrics[:20])

Love Is A Burning Th


In [38]:
detect_langs(all_genres_unique.iloc[0, :].lyrics[20:])

[en:0.9999962044589078]

In [61]:
def get_lang(text):
    try:
        language = detect(text)
    except:
        language = 'not identified'
    return language
all_genres_unique['language'] = all_genres_unique['lyrics'].apply(lambda lyrics : get_lang(lyrics[:200]))

In [62]:
all_genres_unique.loc[all_genres_unique.language == 'not identified']

,index,name,author,link,lyrics,chords,genre,actual_name,language
1750,2126,Hang Em High,Booker T. & the M.G.'s,https://tabs.ultimate-guitar.com/tab/booker-t-...,\r\n \r\n \r\n ...,"['Em', 'D', 'Em', 'D', 'Em', 'D', 'Em', 'A', '...",Jazz,Hang Em High,not identified
2005,2415,Europa,Santana,https://tabs.ultimate-guitar.com/tab/santana/e...,|--------|\r\n|--------|--------|--------|----...,"['Bb7sus4', 'Bb7', 'Ebmaj7', 'Abmaj7', 'G7sus4...",Jazz,Europa,not identified
2295,2798,The Shaker Song,The Manhattan Transfer,https://tabs.ultimate-guitar.com/tab/the-manha...,* * * * * * * * * * * *...,"['G6', 'G11', 'G6', 'G11', 'G6', 'G11', 'G6', ...",Jazz,The Shaker Song,not identified
2409,2947,Igy,Donald Fagen,https://tabs.ultimate-guitar.com/tab/donald-fa...,* * * * * * * * * ...,"['Abm', 'Dbm9', 'Emaj9', 'E/F#', 'Abm', 'Dbm9'...",Jazz,Igy,not identified
2779,3446,La Fiesta,Chick Corea,https://tabs.ultimate-guitar.com/tab/chick-cor...,|--=--=--+--=--=--+--=--=--+--=--=--+--=--=--+...,"['AMaj7', 'Db7', 'DMaj7', 'Ebdim', 'A/E', 'Fdi...",Jazz,La Fiesta,not identified
2828,3510,Three Views Of A Secret,Jaco Pastorius,https://tabs.ultimate-guitar.com/tab/jaco-past...,|-----=-----=-----+-----=-----=-----+-----=---...,"['C#m7', 'Bm7', 'A9', 'F#', 'B9', 'E6', 'G#7',...",Jazz,Three Views Of A Secret,not identified
2857,3547,Wished For You,Squirrel Nut Zippers,https://tabs.ultimate-guitar.com/tab/squirrel-...,\t\t\t ^ ^ ^ ^\r\n,"['Ab7', 'G7', 'Gb7']",Jazz,Wished For You,not identified
3035,3787,Rio Nights,Shakatak,https://tabs.ultimate-guitar.com/tab/shakatak/...,|.... |....\r\n|.... |....\r\n|.... |......,"['Bb/C', 'Fm7', 'Bb/C', 'F7+', 'Bb/C', 'F7+', ...",Jazz,Rio Nights,not identified
3192,4002,Before The Rain,Lee Oskar,https://tabs.ultimate-guitar.com/tab/lee-oskar...,* * * * * * * * * ...,"['C', 'Dm', 'C', 'C', 'Dm', 'C', 'C', 'Dm', 'C...",Jazz,Before The Rain,not identified
7134,8857,Elvis Album,Elvis Presley,https://tabs.ultimate-guitar.com/tab/elvis-pre...,1 + 2 + 3 + 4 + 1 + 2 + 3 + 4 + 1 + 2 + 3 ...,"['A7', 'D7', 'A7', 'E7', 'D7', 'A7', 'A5', 'A5...",Pop,Elvis Album,not identified


In [73]:
all_genres_unique.iloc[1750, :].lyrics

'\xa0 \xa0 \xa0 \xa0 \xa0\r\n\xa0 \xa0 \xa0 \xa0 \xa0\r\n\xa0 \xa0 \xa0 \xa0 \xa0\r\n\xa0 \xa0 \xa0 \xa0 \xa0\r\n\xa0 \xa0 \xa0 \xa0 \xa0\r\n\xa0 \xa0 \xa0 \xa0 \xa0\r\n\xa0 \xa0 \xa0 \xa0 \xa0\r\n\xa0 \xa0 \xa0 \xa0 \xa0\r\n\xa0 \xa0 \xa0 \xa0 \xa0\r\n\xa0 \xa0 \xa0 \xa0 \xa0\r\n\xa0 \xa0 \xa0 \xa0 \xa0\r\n\xa0 \xa0 \xa0 \xa0 \xa0\r\n\xa0 \xa0 \xa0 \xa0 \xa0\r\n\xa0 \xa0 \xa0 \xa0 \xa0\r\n\xa0 \xa0 \xa0 \xa0 \xa0\r\n\xa0 \xa0 \xa0 \xa0 \xa0\r\n\xa0 \xa0 \xa0 \xa0 \xa0\r\n\xa0 \xa0 \xa0 \xa0 \xa0\r\n\xa0 \xa0 \xa0 \xa0 \xa0\r\n'

In [42]:
all_genres_unique.language.value_counts()

en                11775
fr                 1302
it                  649
nl                  556
de                  532
so                  509
af                  486
es                  469
fi                  415
tl                  402
cy                  371
no                  306
not identified      286
sv                  250
pt                  222
id                  217
da                  205
pl                  194
et                  151
ca                  146
sw                  128
tr                   87
ro                   83
sk                   80
hr                   76
sq                   50
sl                   49
ru                   49
cs                   43
vi                   39
hu                   38
lt                   34
lv                   19
bg                    3
ko                    2
uk                    1
Name: language, dtype: int64

In [74]:
all_genres_en = all_genres_unique.loc[all_genres_unique.language.isin(['en'])].drop(columns='index').reset_index().drop(columns='index')
all_genres_en

,name,author,link,lyrics,chords,genre,actual_name,language
0,Ring Of Fire,Johnny Cash,https://tabs.ultimate-guitar.com/tab/johnny-ca...,Love Is A Burning Thing\r\nAnd It Makes A Fier...,"['G', 'C', 'G', 'C', 'G', 'D', 'G', 'D', 'G', ...",Country,Ring Of Fire,en
1,Leaving On A Jet Plane,John Denver,https://tabs.ultimate-guitar.com/tab/john-denv...,"All my bags are packed, I'm ready to go\r\nI'm...","['G', 'C', 'G', 'C', 'G', 'C', 'D', 'G', 'C', ...",Country,Leaving On A Jet Plane,en
2,Hit The Road Jack,Ray Charles,https://tabs.ultimate-guitar.com/tab/ray-charl...,Hit the road Jack and don't you come back no m...,"['Am', 'G', 'F', 'E7', 'Am', 'G', 'F', 'E7', '...",Country,Hit The Road Jack,en
3,The End Of The World,Skeeter Davis,https://tabs.ultimate-guitar.com/tab/skeeter-d...,Why does the sun go on shining\r\nWhy does the...,"['G', 'D', 'Em', 'Bm', 'Am', 'Bm', 'E7', 'Am',...",Country,The End Of The World,en
4,Mama Tried,Merle Haggard,https://tabs.ultimate-guitar.com/tab/merle-hag...,The first thing I remember knowing was a lones...,"['D', 'G', 'D', 'G', 'D', 'A7', 'D', 'G', 'D',...",Country,Mama Tried,en
...,...,...,...,...,...,...,...,...
15928,5150,Machine Gun Kelly,https://tabs.ultimate-guitar.com/tab/machine-g...,Bruises don't heal overnight\r\nI'm a few sips...,"['C', 'G', 'D', 'Em', 'C', 'G', 'D', 'Em', 'C'...",Rock,5150,en
15929,Fake Love Dont Last,Machine Gun Kelly,https://tabs.ultimate-guitar.com/tab/machine-g...,"I watched a movie scene, got déjà vu\r\n'Cause...","['Em', 'G', 'B', 'C', 'Em', 'G', 'B', 'C', 'Em...",Rock,Fake Love Dont Last,en
15930,Sid And Nancy,Machine Gun Kelly,https://tabs.ultimate-guitar.com/tab/machine-g...,I knew a girl who'd wear my t-shirts when she ...,"['D', 'F#m', 'E', 'D', 'F#m', 'E', 'D', 'F#m',...",Rock,Sid And Nancy,en
15931,Forever,Stereophonics,https://tabs.ultimate-guitar.com/tab/stereopho...,Sun beats down on my mind on a friday morning...,"['G', 'C', 'G', 'C', 'G', 'C', 'G', 'C', 'D', ...",Rock,Forever,en
